In [1]:
import os
import re
from itertools import product
from collections import defaultdict

In [2]:
WORKDIR = '.'
SIMULATOR = os.path.join(WORKDIR, '../build/simulator')
SAMPLE_DIR = os.path.join(WORKDIR, '../samples')
SAMPLES = [  # (filename, args)
    ('add', ''),
    ('mul-div', ''),
    ('n!', ''),
    ('simple-fuction', ''),
    ('qsort', ''),
    ('quick_sort', '100000 1 1111'),
    ('ackermann', '3 8'),
    ('matrix_mul', '100 1 1111'),
    ('malloc_free', '50000 1111'),
    ('dhrystone', '200000'),
]
CONFIGS = ['exp2_no_cache.yml', 'exp2_default_cache.yml']

In [4]:
def run_sample(sample, args, config_file):
    config_file = os.path.join(WORKDIR, config_file)
    sample = os.path.join(SAMPLE_DIR, sample)
    with os.popen('{} -c {} {} {}'.format(SIMULATOR, config_file, sample, args)) as result:
        output = '\n'.join(result.readlines())
        try:
            match = re.search(r'instructions=(\d+) cycles=(\d+) CPI=(\d+\.\d+)', output)
            inst_count = int(match.group(1))
            cycles = int(match.group(2))
            cpi = float(match.group(3))
            return inst_count, cycles, cpi
        except:
            print(output)

result = defaultdict(lambda: {})
for config_file in CONFIGS:
    for sample, args in SAMPLES:
        result[sample][config_file] = run_sample(sample, args, config_file)

print('|samples|动态指令数|CPI (no_cache)|CPI (default_cache)|')
print('|-------|---------|--------------|------------------|')
for sample, _ in SAMPLES:
    res = result[sample]
    print('|{}|{}|{}|{}|'.format(sample, res[CONFIGS[0]][0], res[CONFIGS[0]][2], res[CONFIGS[1]][2]))

|samples|动态指令数|CPI (no_cache)|CPI (default_cache)|
|-------|---------|--------------|------------------|
|add|290|125.862|11.91|
|mul-div|312|125.321|11.154|
|n!|295|126.102|11.8|
|simple-fuction|290|125.862|12.772|
|qsort|5397|104.577|1.75|
|quick_sort|18485158|117.571|1.721|
|ackermann|27847017|110.03|1.155|
|matrix_mul|28696552|100.497|1.51|
|malloc_free|68232830|111.482|2.687|
|dhrystone|42847123|108.51|1.335|
